This model uses CNNs to classify audio input into instrument classes.

General Procedure:
	- Use STFTs to transform audio input into spectral graphs
	- Use spectral graphs as "images" to classify into instrument classes

Data Format:
 	- Data is stored in tfrecord form obtained from the Nsynth dataset: https://magenta.tensorflow.org/datasets/nsynth

Based off of aymericdamien's TensorFlow examples: https://github.com/aymericdamien/TensorFlow-Examples

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib.framework.python.ops import audio_ops
from tensorflow.python import debug as tf_debug
import matplotlib.pyplot as plt
import numpy as np

# For debugging on Windows 10
from pyreadline import Readline
readline = Readline()

In [21]:
# Training Parameters
LEARNING_RATE = 0.0001
NUM_STEPS = 5000
BATCH_SIZE = 64

# Network Parameters
NUM_INPUT = 16384   # spectrogram data input (img shape: 128*128)
NUM_CLASSES = 11    # total instrument classes
DROPOUT = 0.25      # Dropout, probability to drop a unit

# Data paths
TRAINING_DATA = ["E:/NSynth/nsynth-train-0.tfrecord", 
                 "E:/NSynth/nsynth-train-1.tfrecord", 
                 "E:/NSynth/nsynth-train-2.tfrecord", 
                 "E:/NSynth/nsynth-train-3.tfrecord",
                 "E:/NSynth/nsynth-train-4.tfrecord",
                 "E:/NSynth/nsynth-train-5.tfrecord",
                 "E:/NSynth/nsynth-train-6.tfrecord",
                 "E:/NSynth/nsynth-train-7.tfrecord",
                 "E:/NSynth/nsynth-train-8.tfrecord",
                 "E:/NSynth/nsynth-train-9.tfrecord",
                 "E:/NSynth/nsynth-train-10.tfrecord"]
TEST_DATA = "E:/NSynth/nsynth-valid.tfrecord"
EVAL_DATA = "E:/NSynth/nsynth-test.tfrecord"
MODEL_PATH = "E:/NSynth/trained_models/"

# Data parameters
SAMPLE_RATE = 16000

In [22]:
# Test GPU presence
# If no errors thrown, GPU is being used
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)

with tf.Session() as sess:
    print (sess.run(c))

[[22. 28.]
 [49. 64.]]


In [23]:
# Create the neural network
def conv_net(x_dict, n_classes, dropout, reuse, is_training):
    
    # Define a scope for reusing the variables
    with tf.variable_scope('ConvNet', reuse=reuse):
        # TF Estimator input is a dict, in case of multiple inputs
        x = x_dict['image']

        # Spectrogram data input is a 1-D vector of 16384 features (128*128 pixels)
        # Reshape to match picture format [Height x Width x Channel]
        # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
        x = tf.reshape(x, shape=[-1, 128, 128, 1])

        # Convolution Layer with 32 filters and a kernel size of 5
        conv1 = tf.layers.conv2d(x, 32, 5, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
        conv1 = tf.layers.max_pooling2d(conv1, 2, 2)

        # Convolution Layer with 64 filters and a kernel size of 3
        conv2 = tf.layers.conv2d(conv1, 64, 3, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
        conv2 = tf.layers.max_pooling2d(conv2, 2, 2)

        # Flatten the data to a 1-D vector for the fully connected layer
        fc1 = tf.contrib.layers.flatten(conv2)

        # Fully connected layer (in tf contrib folder for now)
        fc1 = tf.layers.dense(fc1, 1024)
        # Apply Dropout (if is_training is False, dropout is not applied)
        fc1 = tf.layers.dropout(fc1, rate=dropout, training=is_training)

        # Output layer, class prediction
        out = tf.layers.dense(fc1, n_classes)

    return out

In [24]:
# Define the model function (following TF Estimator Template)
def model_fn(features, labels, mode):
    
    # Build the neural network
    # Because Dropout have different behavior at training and prediction time, we
    # need to create 2 distinct computation graphs that still share the same weights.
    logits_train = conv_net(features, NUM_CLASSES, DROPOUT, reuse=False, is_training=True)
    logits_test = conv_net(features, NUM_CLASSES, DROPOUT, reuse=True, is_training=False)
    
    # Predictions
    pred_classes = tf.argmax(logits_test, axis=1)
    pred_probas = tf.nn.softmax(logits_test)
    
    # If prediction mode, early return
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes) 
        
    # Define loss and optimizer
    labels = tf.reshape(labels, shape=[-1]) # Squeeze labels from 2D to 1D - don't know why they are fed as 2D
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits_train, labels=tf.cast(labels, dtype=tf.int32)))
    optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    
    # Evaluate the accuracy of the model
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)
    
    # TF Estimators requires to return a EstimatorSpec, that specify
    # the different ops for training, evaluating, ...
    estim_specs = tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=pred_classes,
      loss=loss_op,
      train_op=train_op,
      eval_metric_ops={'accuracy': acc_op})

    return estim_specs

In [25]:
# Build the Estimator
model = tf.estimator.Estimator(model_fn, model_dir=MODEL_PATH)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'E:/NSynth/trained_models/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001FC8840B710>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [26]:
# Function to parse the TFRecord
def _parse_(serialized_example):
    feature_list = {
                'audio': tf.FixedLenSequenceFeature(shape=[1], dtype=tf.float32, allow_missing=True),
                'instrument_family': tf.FixedLenSequenceFeature(shape=[], dtype=tf.int64, allow_missing=True),
}
    
    # Extract example by features
    example = tf.parse_single_example(serialized_example, feature_list)
    
    # Convert audio data to normalized spectrogram - time (4sec) vs frequency intensity
    spectrogram = audio_ops.audio_spectrogram(example["audio"], window_size=1024, stride=64)
    max_val = tf.reduce_max(spectrogram, axis=None)
    min_const = tf.constant(255.)
    brightness_const = tf.divide(min_const, max_val)   # Calculate normalization constant
    brightened_spect = tf.multiply(spectrogram, brightness_const)
    minned_spectrogram = tf.minimum(brightened_spect, min_const) # Remove other spikes?
    expanded = tf.expand_dims(minned_spectrogram, -1)
    resized = tf.image.resize_bilinear(expanded, [128, 128])
    squeezed = tf.squeeze(resized, 0)
    flipped = tf.image.flip_left_right(squeezed)
    normalized_spectrogram = tf.image.transpose_image(flipped)
    
    # Cast data to input format required by model
    image = normalized_spectrogram
    label = example['instrument_family']
    return (dict({'image':image}),label)



# Define the data input function for training
def tfrecord_train_input_fn(batch_size=BATCH_SIZE):
    files = tf.data.Dataset.from_tensor_slices(TRAINING_DATA)
    tfrecord_dataset = files.interleave(lambda x: tf.data.TFRecordDataset(x), cycle_length=11, block_length=1)
    tfrecord_dataset = tfrecord_dataset.map(lambda   x:_parse_(x)).shuffle(True).batch(batch_size)
    tfrecord_iterator = tfrecord_dataset.make_one_shot_iterator()
    
    return tfrecord_iterator.get_next()

# Train the Model
# With debug
# hooks = [tf_debug.LocalCLIDebugHook(ui_type="readline")]
# model.train(tfrecord_train_input_fn, steps=NUM_STEPS, hooks=hooks)
# without debug
model.train(tfrecord_train_input_fn, steps=NUM_STEPS)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into E:/NSynth/trained_models/model.ckpt.
INFO:tensorflow:loss = 3.8422403, step = 0
INFO:tensorflow:global_step/sec: 1.04266
INFO:tensorflow:loss = 1.5114232, step = 100 (95.910 sec)
INFO:tensorflow:global_step/sec: 1.04947
INFO:tensorflow:loss = 0.89906955, step = 200 (95.350 sec)
INFO:tensorflow:global_step/sec: 1.04596
INFO:tensorflow:loss = 1.1444888, step = 300 (95.543 sec)
INFO:tensorflow:global_step/sec: 1.05481
INFO:tensorflow:loss = 0.8306154, step = 400 (94.857 sec)
INFO:tensorflow:global_step/sec: 1.04974
INFO:tensorflow:loss = 0.54740345, step = 500 (95.208 sec)
INFO:tensorflow:global_step/sec: 1.05231
INFO:tensorflow:loss = 0.45489484, step = 600 (95.084 sec)
INFO:tensorflow:Saving checkpoints for 6

In [29]:
# Evaluate the Model
# Define the input function for evaluating
def tfrecord_eval_input_fn(batch_size=BATCH_SIZE):
    tfrecord_dataset = tf.data.TFRecordDataset(EVAL_DATA)
    tfrecord_dataset = tfrecord_dataset.map(lambda   x:_parse_(x)).shuffle(True).batch(batch_size)
    tfrecord_iterator = tfrecord_dataset.make_one_shot_iterator()
    return tfrecord_iterator.get_next()


# Use the Estimator 'evaluate' method
model.evaluate(tfrecord_eval_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-03-21:56:54
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from E:/NSynth/trained_models/model.ckpt-4519
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-03-21:57:46
INFO:tensorflow:Saving dict for global step 4519: accuracy = 0.18261719, global_step = 4519, loss = 13.617434
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 4519: E:/NSynth/trained_models/model.ckpt-4519


{'accuracy': 0.18261719, 'loss': 13.617434, 'global_step': 4519}

In [30]:
# Epoch 2
model.train(tfrecord_train_input_fn, steps=NUM_STEPS)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from E:/NSynth/trained_models/model.ckpt-4519
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 4519 into E:/NSynth/trained_models/model.ckpt.
INFO:tensorflow:loss = 4.3638897, step = 4519
INFO:tensorflow:global_step/sec: 1.0699
INFO:tensorflow:loss = 0.79245895, step = 4619 (93.469 sec)
INFO:tensorflow:global_step/sec: 1.06951
INFO:tensorflow:loss = 0.8431303, step = 4719 (93.537 sec)
INFO:tensorflow:global_step/sec: 1.06844
INFO:tensorflow:loss = 0.95865005, step = 4819 (93.557 sec)
INFO:tensorflow:global_step/sec: 1.07369
INFO:tensorflow:loss = 0.5191035, step = 4919 (93.168 sec)
INFO:tensorflow:global_step/sec: 1.07231
INFO:tensorflow:loss = 0.43454367, step = 5019 (93.226 sec)
INFO:tensorflow:global_step/sec: 1.06185
INFO:

In [31]:
# Evaluate the Model
# Define the input function for evaluating
def tfrecord_valid_input_fn(batch_size=BATCH_SIZE):
    tfrecord_dataset = tf.data.TFRecordDataset(TEST_DATA)
    tfrecord_dataset = tfrecord_dataset.map(lambda   x:_parse_(x)).shuffle(True).batch(batch_size)
    tfrecord_iterator = tfrecord_dataset.make_one_shot_iterator()
    return tfrecord_iterator.get_next()


# Use the Estimator 'evaluate' method
model.evaluate(tfrecord_valid_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-04-01:24:50
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from E:/NSynth/trained_models/model.ckpt-9038
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-04-01:27:26
INFO:tensorflow:Saving dict for global step 9038: accuracy = 0.22677079, global_step = 9038, loss = 4.638931
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 9038: E:/NSynth/trained_models/model.ckpt-9038


{'accuracy': 0.22677079, 'loss': 4.638931, 'global_step': 9038}

In [32]:
# Epoch 3
model.train(tfrecord_train_input_fn, steps=NUM_STEPS)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from E:/NSynth/trained_models/model.ckpt-9038
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 9038 into E:/NSynth/trained_models/model.ckpt.
INFO:tensorflow:loss = 3.6084359, step = 9038
INFO:tensorflow:global_step/sec: 0.921273
INFO:tensorflow:loss = 1.0256901, step = 9138 (108.591 sec)
INFO:tensorflow:global_step/sec: 0.926213
INFO:tensorflow:loss = 0.5045406, step = 9238 (107.921 sec)
INFO:tensorflow:global_step/sec: 0.9189
INFO:tensorflow:loss = 0.6528921, step = 9338 (108.860 sec)
INFO:tensorflow:global_step/sec: 0.922095
INFO:tensorflow:loss = 0.6241628, step = 9438 (108.415 sec)
INFO:tensorflow:global_step/sec: 0.919629
INFO:tensorflow:loss = 0.29207772, step = 9538 (108.783 sec)
INFO:tensorflow:Saving checkpoints for 

In [33]:
# Use the Estimator 'evaluate' method
model.evaluate(tfrecord_valid_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-04-02:44:50
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from E:/NSynth/trained_models/model.ckpt-13557
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-04-02:47:26
INFO:tensorflow:Saving dict for global step 13557: accuracy = 0.28056476, global_step = 13557, loss = 3.7641368
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 13557: E:/NSynth/trained_models/model.ckpt-13557


{'accuracy': 0.28056476, 'loss': 3.7641368, 'global_step': 13557}

In [34]:
# Epoch 4
model.train(tfrecord_train_input_fn, steps=NUM_STEPS)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from E:/NSynth/trained_models/model.ckpt-13557
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 13557 into E:/NSynth/trained_models/model.ckpt.
INFO:tensorflow:loss = 2.806211, step = 13557
INFO:tensorflow:global_step/sec: 1.0657
INFO:tensorflow:loss = 0.4693267, step = 13657 (93.836 sec)
INFO:tensorflow:global_step/sec: 1.06826
INFO:tensorflow:loss = 0.36490557, step = 13757 (93.652 sec)
INFO:tensorflow:global_step/sec: 1.06103
INFO:tensorflow:loss = 0.55630183, step = 13857 (94.206 sec)
INFO:tensorflow:global_step/sec: 1.05955
INFO:tensorflow:loss = 0.40627837, step = 13957 (94.420 sec)
INFO:tensorflow:global_step/sec: 1.06474
INFO:tensorflow:loss = 0.22132756, step = 14057 (93.880 sec)
INFO:tensorflow:global_step/sec: 1.066

In [35]:
# Use the Estimator 'evaluate' method
model.evaluate(tfrecord_valid_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-04-14:26:07
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from E:/NSynth/trained_models/model.ckpt-18076
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-04-14:28:43
INFO:tensorflow:Saving dict for global step 18076: accuracy = 0.18291529, global_step = 18076, loss = 33.829628
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 18076: E:/NSynth/trained_models/model.ckpt-18076


{'accuracy': 0.18291529, 'loss': 33.829628, 'global_step': 18076}

In [36]:
# Epoch 5
model.train(tfrecord_train_input_fn, steps=NUM_STEPS)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from E:/NSynth/trained_models/model.ckpt-18076
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 18076 into E:/NSynth/trained_models/model.ckpt.
INFO:tensorflow:loss = 14.736004, step = 18076
INFO:tensorflow:global_step/sec: 1.05756
INFO:tensorflow:loss = 0.39310628, step = 18176 (94.557 sec)
INFO:tensorflow:global_step/sec: 1.06601
INFO:tensorflow:loss = 0.21003126, step = 18276 (93.848 sec)
INFO:tensorflow:global_step/sec: 1.06449
INFO:tensorflow:loss = 0.38961357, step = 18376 (93.901 sec)
INFO:tensorflow:global_step/sec: 1.06315
INFO:tensorflow:loss = 0.25020614, step = 18476 (94.106 sec)
INFO:tensorflow:global_step/sec: 1.05887
INFO:tensorflow:loss = 0.2597365, step = 18576 (94.395 sec)
INFO:tensorflow:global_step/sec: 1.0

In [37]:
# Use the Estimator 'evaluate' method
model.evaluate(tfrecord_valid_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-04-20:42:34
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from E:/NSynth/trained_models/model.ckpt-22595
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-04-20:45:10
INFO:tensorflow:Saving dict for global step 22595: accuracy = 0.22156492, global_step = 22595, loss = 9.878408
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 22595: E:/NSynth/trained_models/model.ckpt-22595


{'accuracy': 0.22156492, 'loss': 9.878408, 'global_step': 22595}

In [38]:
# Epoch 6
model.train(tfrecord_train_input_fn, steps=NUM_STEPS)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from E:/NSynth/trained_models/model.ckpt-22595
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 22595 into E:/NSynth/trained_models/model.ckpt.
INFO:tensorflow:loss = 6.800639, step = 22595
INFO:tensorflow:global_step/sec: 1.05812
INFO:tensorflow:loss = 0.24657896, step = 22695 (94.508 sec)
INFO:tensorflow:global_step/sec: 1.06674
INFO:tensorflow:loss = 0.11622308, step = 22795 (93.797 sec)
INFO:tensorflow:global_step/sec: 1.06462
INFO:tensorflow:loss = 0.34380636, step = 22895 (93.877 sec)
INFO:tensorflow:global_step/sec: 1.06851
INFO:tensorflow:loss = 0.13160962, step = 22995 (93.621 sec)
INFO:tensorflow:global_step/sec: 1.06553
INFO:tensorflow:loss = 0.2372436, step = 23095 (93.818 sec)
INFO:tensorflow:global_step/sec: 1.07

In [39]:
# Use the Estimator 'evaluate' method
model.evaluate(tfrecord_valid_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-04-22:26:05
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from E:/NSynth/trained_models/model.ckpt-27114
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-04-22:28:40
INFO:tensorflow:Saving dict for global step 27114: accuracy = 0.20965452, global_step = 27114, loss = 10.410645
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 27114: E:/NSynth/trained_models/model.ckpt-27114


{'accuracy': 0.20965452, 'loss': 10.410645, 'global_step': 27114}

In [40]:
# Use the Estimator 'evaluate' method
model.evaluate(tfrecord_eval_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-04-23:37:09
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from E:/NSynth/trained_models/model.ckpt-27114
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-04-23:38:00
INFO:tensorflow:Saving dict for global step 27114: accuracy = 0.20092773, global_step = 27114, loss = 10.540672
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 27114: E:/NSynth/trained_models/model.ckpt-27114


{'accuracy': 0.20092773, 'loss': 10.540672, 'global_step': 27114}